In [1]:
!pip install geocoder

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 kB 4.7 MB/s eta 0:00:00


In [2]:
import pandas as pd

DATA = '/kaggle/input/global-metropolis-weather-dataset/weather_data_extended.csv'

df = pd.read_csv(filepath_or_buffer=DATA, parse_dates=['Observation Time'])

df.head()

,Location,Observation Time,Temperature (°C),Feels Like (°C),Humidity (%),Wind Speed (kph),Weather Condition,Cloud Cover (%),Pressure (mb),UV Index,Visibility (km)
0,New York,2024-04-05 14:15:00,8.3,4.5,40,29.9,Partly cloudy,75,1003,2,16
1,New York,2024-04-05 14:15:00,8.3,4.5,40,29.9,Partly cloudy,75,1003,2,16
2,New York,2024-04-05 14:15:00,8.3,4.5,40,29.9,Partly cloudy,75,1003,2,16
3,New York,2024-04-05 14:15:00,8.3,4.5,40,29.9,Partly cloudy,75,1003,2,16
4,New York,2024-04-05 14:15:00,8.3,4.5,40,29.9,Partly cloudy,75,1003,2,16


It looks like we have some duplicate rows. Let's drop them.

In [3]:
df = df.drop_duplicates(ignore_index=True)

Let's geolocate and draw a map and be done.

In [4]:
import arrow
from geocoder import arcgis

time_start = arrow.now()
df['latlng'] = df['Location'].apply(func=lambda x: arcgis(x).latlng)
df['latitude'] = df['latlng'].apply(func=lambda x: x[0])
df['longitude'] = df['latlng'].apply(func=lambda x: x[1])

print('{} done'.format(arrow.now() - time_start))


0:00:04.552549 done


We don't have a lot of rows, but we have a fair number of columns; let's make maps.

In [5]:
import warnings
from plotly import express

warnings.filterwarnings(action='ignore', category=FutureWarning)
for column in ['Temperature (°C)', 'Feels Like (°C)', 'Humidity (%)', 'Wind Speed (kph)', 'Weather Condition', 'Cloud Cover (%)',
               'Pressure (mb)', 'UV Index', 'Visibility (km)']:
    express.scatter_mapbox(mapbox_style='open-street-map', data_frame=df, lat='latitude', lon='longitude', zoom=1, hover_name='Location',
                           color=column).show()